# Evidential NN

## Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision.datasets.mnist import MNIST
import torchvision.transforms as transforms

import scipy.ndimage as nd
from PIL import Image

import numpy as np
from matplotlib import pyplot as plt

import os
import json
import copy
from time import time
from collections import OrderedDict
from collections import defaultdict

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import SubsetRandomSampler, ConcatDataset

In [ ]:
! mkdir results

## Dempster-Shafer layers

Credit to **paul-bd** for their implementation of the evidential layers:

https://github.com/paul-bd/DempsterShaferTheory

In [ ]:
class Distance_layer(torch.nn.Module):
    '''
    verified
    '''
    def __init__(self, n_prototypes, n_feature_maps):
        super(Distance_layer, self).__init__()
        self.w = torch.nn.Linear(in_features=n_feature_maps, out_features=n_prototypes, bias=False).weight
        self.n_prototypes = n_prototypes

    def forward(self, inputs):
        for i in range(self.n_prototypes):
            if i == 0:
                un_mass_i = (self.w[i, :] - inputs) ** 2
                un_mass_i = torch.sum(un_mass_i, dim=-1, keepdim=True)
                un_mass = un_mass_i

            if i >= 1:
                un_mass_i = (self.w[i, :] - inputs) ** 2
                un_mass_i = torch.sum(un_mass_i, dim=-1, keepdim=True)
                un_mass = torch.cat([un_mass, un_mass_i], -1)
        return un_mass


class DistanceActivation_layer(torch.nn.Module):
    '''
    verified
    '''
    def __init__(self, n_prototypes,init_alpha=0,init_gamma=0.1):
        super(DistanceActivation_layer, self).__init__()
        self.eta = torch.nn.Linear(in_features=n_prototypes, out_features=1, bias=False)#.weight.data.fill_(torch.from_numpy(np.array(init_gamma)).to(device))
        self.xi = torch.nn.Linear(in_features=n_prototypes, out_features=1, bias=False)#.weight.data.fill_(torch.from_numpy(np.array(init_alpha)).to(device))
        #torch.nn.init.kaiming_uniform_(self.eta.weight)
        #torch.nn.init.kaiming_uniform_(self.xi.weight)
        torch.nn.init.constant_(self.eta.weight,init_gamma)
        torch.nn.init.constant_(self.xi.weight,init_alpha)
        #self.alpha_test = 1/(torch.exp(-self.xi.weight)+1)
        self.n_prototypes = n_prototypes
        self.alpha = None

    def forward(self, inputs):
        gamma=torch.square(self.eta.weight)
        alpha=torch.neg(self.xi.weight)
        alpha=torch.exp(alpha)+1
        alpha=torch.div(1, alpha)
        self.alpha=alpha
        si=torch.mul(gamma, inputs)
        si=torch.neg(si)
        si=torch.exp(si)
        si = torch.mul(si, alpha)
        max_val, max_idx = torch.max(si, dim=-1, keepdim=True)
        si /= (max_val + 0.0001)

        return si


'''class Belief_layer(torch.nn.Module):
    def __init__(self, prototypes, num_class):
        super(DS2, self).__init__()
        self.beta = torch.nn.Linear(in_features=prototypes, out_features=num_class, bias=False).weight
        self.prototypes = prototypes
        self.num_class = num_class
    def forward(self, inputs):
        beta = torch.square(self.beta)
        beta_sum = torch.sum(beta, dim=0, keepdim=True)
        self.u = torch.div(beta, beta_sum)
        inputs_new = torch.unsqueeze(inputs, dim=-2)
        for i in range(self.prototypes):
            if i == 0:
                mass_prototype_i = torch.mul(self.u[:, i], inputs_new[..., i])  #batch_size * n_class
                mass_prototype = torch.unsqueeze(mass_prototype_i, -2)
            if i > 0:
                mass_prototype_i = torch.unsqueeze(torch.mul(self.u[:, i], inputs_new[..., i]), -2)
                mass_prototype = torch.cat([mass_prototype, mass_prototype_i], -2)
        return mass_prototype'''

class Belief_layer(torch.nn.Module):
    '''
    verified
    '''
    def __init__(self, n_prototypes, num_class):
        super(Belief_layer, self).__init__()
        self.beta = torch.nn.Linear(in_features=n_prototypes, out_features=num_class, bias=False).weight
        self.num_class = num_class
    def forward(self, inputs):
        beta = torch.square(self.beta)
        beta_sum = torch.sum(beta, dim=0, keepdim=True)
        u = torch.div(beta, beta_sum)
        mass_prototype = torch.einsum('cp,b...p->b...pc',u, inputs)
        return mass_prototype

class Omega_layer(torch.nn.Module):
    '''
    verified, give same results
    '''
    def __init__(self, n_prototypes, num_class):
        super(Omega_layer, self).__init__()
        self.n_prototypes = n_prototypes
        self.num_class = num_class

    def forward(self, inputs):
        mass_omega_sum = 1 - torch.sum(inputs, -1, keepdim=True)
        #mass_omega_sum = 1. - mass_omega_sum[..., 0]
        #mass_omega_sum = torch.unsqueeze(mass_omega_sum, -1)
        mass_with_omega = torch.cat([inputs, mass_omega_sum], -1)
        return mass_with_omega

class Dempster_layer(torch.nn.Module):
    '''
    verified give same results
    '''
    def __init__(self, n_prototypes, num_class):
        super(Dempster_layer, self).__init__()
        self.n_prototypes = n_prototypes
        self.num_class = num_class

    def forward(self, inputs):
        m1 = inputs[..., 0, :]
        omega1 = torch.unsqueeze(inputs[..., 0, -1], -1)
        for i in range(self.n_prototypes - 1):
            m2 = inputs[..., (i + 1), :]
            omega2 = torch.unsqueeze(inputs[..., (i + 1), -1], -1)
            combine1 = torch.mul(m1, m2)
            combine2 = torch.mul(m1, omega2)
            combine3 = torch.mul(omega1, m2)
            combine1_2 = combine1 + combine2
            combine2_3 = combine1_2 + combine3
            combine2_3 = combine2_3 / torch.sum(combine2_3, dim=-1, keepdim=True)
            m1 = combine2_3
            omega1 = torch.unsqueeze(combine2_3[..., -1], -1)
        return m1


class DempsterNormalize_layer(torch.nn.Module):
    '''
    verified
    '''
    def __init__(self):
        super(DempsterNormalize_layer, self).__init__()
    def forward(self, inputs):
        mass_combine_normalize = inputs / torch.sum(inputs, dim=-1, keepdim=True)
        return mass_combine_normalize


class Dempster_Shafer_module(torch.nn.Module):
    def __init__(self, n_feature_maps, n_classes, n_prototypes):
        super(Dempster_Shafer_module, self).__init__()
        self.n_prototypes = n_prototypes
        self.n_classes = n_classes
        self.n_feature_maps = n_feature_maps
        self.ds1 = Distance_layer(n_prototypes=self.n_prototypes, n_feature_maps=self.n_feature_maps)
        self.ds1_activate = DistanceActivation_layer(n_prototypes = self.n_prototypes)
        self.ds2 = Belief_layer(n_prototypes= self.n_prototypes, num_class=self.n_classes)
        self.ds2_omega = Omega_layer(n_prototypes= self.n_prototypes,num_class= self.n_classes)
        self.ds3_dempster = Dempster_layer(n_prototypes= self.n_prototypes,num_class= self.n_classes)
        self.ds3_normalize = DempsterNormalize_layer()

    def forward(self, inputs):
        '''
        '''
        ED = self.ds1(inputs)
        ED_ac = self.ds1_activate(ED)
        mass_prototypes = self.ds2(ED_ac)
        mass_prototypes_omega = self.ds2_omega(mass_prototypes)
        mass_Dempster = self.ds3_dempster(mass_prototypes_omega)
        mass_Dempster_normalize = self.ds3_normalize(mass_Dempster)
        return mass_Dempster_normalize



def tile(a, dim, n_tile, device):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)])).to(
        device)
    return torch.index_select(a, dim, order_index)


class DM(torch.nn.Module):
    def __init__(self, num_class, nu=0.9, device=torch.device('cpu')):
        super(DM, self).__init__()
        self.nu = nu
        self.num_class = num_class
        self.device = device

    def forward(self, inputs):
        upper = torch.unsqueeze((1 - self.nu) * inputs[..., -1], -1)  # here 0.1 = 1 - \nu
        upper = tile(upper, dim=-1, n_tile=self.num_class + 1, device=self.device)
        outputs = (inputs + upper)[..., :-1]
        return outputs



## Data loading

In [ ]:
BATCH_SIZE = 1000

In [ ]:
data_full = MNIST("./data/mnist",
                   download=True,
                   train=True,
                   transform=transforms.Compose([transforms.ToTensor()]))

data_train, data_corrections = torch.utils.data.random_split(
    data_full, [30000, 30000])

data_val = MNIST("./data/mnist",
                 train=False,
                 download=True,
                 transform=transforms.Compose([transforms.ToTensor()]))

dataloader_train = DataLoader(
    data_train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_corrections = DataLoader(
    data_corrections, batch_size=BATCH_SIZE, shuffle=True)
dataloader_val = DataLoader(data_val, batch_size=BATCH_SIZE)

BATCH_PRINT = len(data_train) // BATCH_SIZE // 3

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



## Utility functions

In [ ]:
def get_device():
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    return device

def string_time(elapsed):
    return "%im %is" %(int(elapsed / 60), int(elapsed % 60))

def calc_loss(prediction, labels): 
    l = criterion(prediction, labels)
    return l.float()

In [ ]:
device = get_device()
criterion = torch.nn.CrossEntropyLoss()

## Training cycle

In [ ]:
def train_epoch(model, optimizer):
    # ======== training ========
    print('Training...\n')

    model.train()
    run_loss = []
    preds = []
    tgts = []
    
    for step, batch in enumerate(train_dataloader):
        input, labels = batch[0].to(device), batch[1].to(device)

        if step % BATCH_PRINT == 0 and not step == 0:
            print('  Batch %i  of  %i;\tElapsed time: ' %(step + 1,
                len(train_dataloader)) + string_time(time() - start))
            
        optimizer.zero_grad()
        
        output = model(input)
        loss = calc_loss(output, labels)

        pred = output.data.cpu().argmax(dim=1).squeeze()
        preds += pred.tolist()
        tgts += labels.tolist()

        loss.backward()
        optimizer.step()
        
        run_loss.append(loss.item())

    run_acc = np.mean(np.array(preds) == np.array(tgts))

    return run_loss, run_acc


def evaluate_model(model):
    model.eval()
    run_loss = []
    preds = []
    tgts = []

    for step, batch in enumerate(val_dataloader):
        input, labels = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(input)
            loss = calc_loss(output, labels)

            pred = output.data.cpu().argmax(dim=1).squeeze()

            preds += pred.tolist()
            tgts += labels.tolist()

        run_loss.append(loss.item())
    
    val_acc = np.mean(np.array(preds) == np.array(tgts))

    return np.array(run_loss).mean(), val_acc


def train_model(model, optimizer, num_epochs=25, name='model', 
                continuing=False, epoch_save=5):

    dir = "./models/" + name + "/"
    if not os.path.exists("./models/"):
        os.mkdir("./models/")
    if not os.path.exists(dir):
        os.mkdir(dir)
        
    global start, train_history, val_history
    
    if not continuing:
        train_history = []
        val_history = []
    
    start = time()

    for epoch in range(num_epochs):
        
        print('\n======= Epoch %i / %i =======' %(epoch + 1, num_epochs))
        
        run_loss, run_acc = train_epoch(model, optimizer)
        train_history.append(np.array(run_loss))
        
        print("\n  Training statistics:   \tCE: %.3f, Accuracy: %.3f"  
              %(train_history[-1].mean(), run_acc))
        
        # ======== validating ========
        print("\nValidating...")

        val_loss, val_acc = evaluate_model(model)
        val_history.append(val_loss)

        print("\n  Validation statistics:\tCE: %.3f, Accuracy: %.3f"  
              %(val_history[-1].mean(), val_acc))

        # ======== logging ========
        #plot_history(train_history, val_history)

        if (epoch + 1) % epoch_save == 0:
            torch.save(model.state_dict(), 
                       dir + "checkpoint" + str(epoch + 1) + "_" + name)

        #np.save(dir + 'train_logs' + "_" + name, np.array(train_history))
        #np.save(dir + 'val_logs' + "_" + name, np.array(val_history))
        
        print("  Elapsed time: " + string_time(time() - start))
    
    return model

## Models

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=32, 
                            kernel_size=1, padding=1), 
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),

            torch.nn.Conv2d(in_channels=32, out_channels=64, 
                            kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),
                
            torch.nn.Conv2d(in_channels=64, out_channels=64, 
                            kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),

            torch.nn.Flatten(),
            torch.nn.Linear(576, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 100)
        )
  
    def forward(self, x):
        return self.layers(x)


class FCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(28*28, 500) 
        self.linear2 = nn.Linear(500, 500) 
        self.linear2 = nn.Linear(500, 100) 

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        return x

class Lin_tail(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(100, 10)

    def forward(self, x, return_mass=False):
        x = self.fc(x)
        return x

class DS_tail(nn.Module):
    def __init__(self):
        super().__init__()
        self.ds = Dempster_Shafer_module(100, 10, 200)
        self.dm = DM(10, nu=1, device=device)

    def forward(self, x, return_mass=False):
        mass = self.ds(x)
        if return_mass:
            return mass
        
        return self.dm(mass)

class Model(nn.Module):
    def __init__(self, head, tail):
        super().__init__()
        self.head = head
        self.tail = tail

    def forward(self, x, return_mass=False):
        x = self.head(x)
        return self.tail(x, return_mass)

In [ ]:
num_classes = 10
num_epochs = 20
device = get_device()

# Training

We start by 
 

*   training the model for N=20 epochs with a probabilistic head, and then
*   fine-tuning it for M=100 epochs with a Dempster-Shafer head



### CNN

In [ ]:
train_dataloader = dataloader_train
val_dataloader = dataloader_val

In [ ]:
cnn = CNN()
linear = Lin_tail()

model = Model(cnn, linear)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.005)
model = train_model(model, optimizer, num_epochs=20, name="CNN_lin")


======= Epoch 1 / 20 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 1s
  Batch 21  of  30;	Elapsed time: 0m 2s

  Training statistics:   	CE: 2.300

Validating...

  Validation statistics:	CE: 2.292, Accuracy: 0.321
  Elapsed time: 0m 4s

======= Epoch 2 / 20 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 5s
  Batch 21  of  30;	Elapsed time: 0m 6s

  Training statistics:   	CE: 2.164

Validating...

  Validation statistics:	CE: 1.888, Accuracy: 0.630
  Elapsed time: 0m 8s

======= Epoch 3 / 20 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 9s
  Batch 21  of  30;	Elapsed time: 0m 10s

  Training statistics:   	CE: 1.842

Validating...

  Validation statistics:	CE: 1.734, Accuracy: 0.737
  Elapsed time: 0m 12s

======= Epoch 4 / 20 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 13s
  Batch 21  of  30;	Elapsed time: 0m 14s

  Training statistics:   	CE: 1.740

Validating...

  Validation statistics:	CE: 1.615, Accuracy: 0.869
  Elapsed time: 0m

In [ ]:
cnn = CNN()
linear = Lin_tail()

model = Model(cnn, linear)
model.load_state_dict(torch.load('./models/CNN_lin/checkpoint20_CNN_lin'))

<All keys matched successfully>

In [ ]:
ds = DS_tail()

model = Model(cnn, ds)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
model = train_model(model, optimizer, num_epochs=100, name="CNN")


======= Epoch 1 / 100 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 2s
  Batch 21  of  30;	Elapsed time: 0m 5s

  Training statistics:   	CE: 2.238, Accuracy: 0.113

Validating...

  Validation statistics:	CE: 2.214, Accuracy: 0.114
  Elapsed time: 0m 9s

======= Epoch 2 / 100 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 12s
  Batch 21  of  30;	Elapsed time: 0m 14s

  Training statistics:   	CE: 2.211, Accuracy: 0.120

Validating...

  Validation statistics:	CE: 2.209, Accuracy: 0.114
  Elapsed time: 0m 18s

======= Epoch 3 / 100 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 20s
  Batch 21  of  30;	Elapsed time: 0m 23s

  Training statistics:   	CE: 2.199, Accuracy: 0.185

Validating...

  Validation statistics:	CE: 2.169, Accuracy: 0.202
  Elapsed time: 0m 26s

======= Epoch 4 / 100 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 29s
  Batch 21  of  30;	Elapsed time: 0m 31s

  Training statistics:   	CE: 2.177, Accuracy: 0.190

Validatin

In [ ]:
cnn = CNN()
ds = DS_tail()

model = Model(cnn, ds)
model.load_state_dict(torch.load('./models/CNN/checkpoint100_CNN'))

### FCN

In [ ]:
fcn = FCN()
linear = Lin_tail()

model = Model(fcn, linear)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.005)
model = train_model(model, optimizer, num_epochs=20, name="FCN_lin")


======= Epoch 1 / 20 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 0s
  Batch 21  of  30;	Elapsed time: 0m 1s

  Training statistics:   	CE: 2.197, Accuracy: 0.413

Validating...

  Validation statistics:	CE: 1.945, Accuracy: 0.581
  Elapsed time: 0m 3s

======= Epoch 2 / 20 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 4s
  Batch 21  of  30;	Elapsed time: 0m 5s

  Training statistics:   	CE: 1.786, Accuracy: 0.748

Validating...

  Validation statistics:	CE: 1.692, Accuracy: 0.821
  Elapsed time: 0m 7s

======= Epoch 3 / 20 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 8s
  Batch 21  of  30;	Elapsed time: 0m 9s

  Training statistics:   	CE: 1.688, Accuracy: 0.824

Validating...

  Validation statistics:	CE: 1.674, Accuracy: 0.834
  Elapsed time: 0m 10s

======= Epoch 4 / 20 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 12s
  Batch 21  of  30;	Elapsed time: 0m 12s

  Training statistics:   	CE: 1.662, Accuracy: 0.857

Validating...

  V

In [ ]:
fcn = FCN()
linear = Lin_tail()

model = Model(fcn, linear)
model.load_state_dict(torch.load('./models/FCN_lin/checkpoint20_FCN_lin'))

<All keys matched successfully>

In [ ]:
ds = DS_tail()

model = Model(fcn, ds)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
model = train_model(model, optimizer, num_epochs=100, name="FCN")


======= Epoch 1 / 100 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 2s
  Batch 21  of  30;	Elapsed time: 0m 4s

  Training statistics:   	CE: 2.245, Accuracy: 0.182

Validating...

  Validation statistics:	CE: 2.215, Accuracy: 0.202
  Elapsed time: 0m 8s

======= Epoch 2 / 100 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 10s
  Batch 21  of  30;	Elapsed time: 0m 12s

  Training statistics:   	CE: 2.169, Accuracy: 0.250

Validating...

  Validation statistics:	CE: 2.115, Accuracy: 0.291
  Elapsed time: 0m 16s

======= Epoch 3 / 100 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 18s
  Batch 21  of  30;	Elapsed time: 0m 20s

  Training statistics:   	CE: 2.092, Accuracy: 0.291

Validating...

  Validation statistics:	CE: 2.068, Accuracy: 0.298
  Elapsed time: 0m 24s

======= Epoch 4 / 100 =======
Training...

  Batch 11  of  30;	Elapsed time: 0m 26s
  Batch 21  of  30;	Elapsed time: 0m 29s

  Training statistics:   	CE: 2.045, Accuracy: 0.395

Validatin

In [ ]:
fcn = FCN()
ds = DS_tail()

model = Model(fcn, ds)
model.load_state_dict(torch.load('./models/FCN/checkpoint100_FCN'))

# Correction

## Load models

Reload models as we do not need GPU anymore:

In [ ]:
fcn = FCN()
ds = DS_tail()

fcn = Model(fcn, ds)
fcn.load_state_dict(torch.load('./checkpoint100_FCN', 
                               map_location=torch.device('cpu')))

cnn = CNN()
ds = DS_tail()

cnn = Model(cnn, ds)
cnn.load_state_dict(torch.load('./checkpoint100_CNN',
                               map_location=torch.device('cpu')))

<All keys matched successfully>

## Correction functions

Correction functions on contour functions as defined by Mutmainah et al

(https://link.springer.com/chapter/10.1007/978-3-031-17801-6_11)

In [1]:
def CD_correction(beta, pl):
    return 1 - (1 - pl) * beta[None, :]

def CR_correction(beta, pl):
    return pl * beta[None, :]

def CN_correction(beta, pl):
    return 0.5 + (pl - 0.5) * (2 * beta[None, :] - 1)

def no_correction(beta, pl):
    return pl


def calc_epl(bpl, labels):
    correct_matr = torch.zeros_like(bpl)
    correct_matr[torch.arange(len(labels)), labels] = 1  

    diff = bpl - correct_matr
    return torch.mul(diff, diff).sum()

## Utility



*   A method to retrieve fuzzy prediction from mass 
*   And calculation of the Utility function

Credit to **tongzheng1992** for their jupyter notebook

(https://github.com/tongzheng1992/E-CNN-classifier)



In [ ]:
num_class = len(data_full.classes)

In [ ]:
import math
from scipy.optimize import minimize


# aim func: cross entropy
def func(x):
    fun=0
    for i in range(len(x)):
        fun += x[i] * math.log10(x[i])
    return fun

#constraint 1: the sum of weights is 1
def cons1(x):
    return sum(x)

#constraint 2: define tolerance to imprecision
def cons2(x):
    tol = 0
    for i in range(len(x)):
        tol += (len(x) -(i+1)) * x[i] / (len(x) - 1)
    return tol

def PowerSetsBinary(items):  
    #generate all combination of N items  
    N = len(items)  
    #enumerate the 2**N possible combinations  
    set_all=[]
    for i in range(2**N):
        combo = []  
        for j in range(N):  
            if(i >> j ) % 2 == 1:  
                combo.append(items[j]) 
        set_all.append(combo)
    return set_all

for j in range(2, (num_class + 1)):
    num_weights = j
    ini_weights = np.asarray(np.random.rand(num_weights))

    name='weight'+str(j)
    locals()['weight'+str(j)]= np.zeros([5, j])

    for i in range(5):
        tol = 0.5 + i * 0.1

        cons = ({'type': 'eq', 'fun' : lambda x: cons1(x)-1},
            {'type': 'eq', 'fun' : lambda x: cons2(x)-tol},
            {'type': 'ineq', 'fun' : lambda x: x-0.00000001}
            )
    
        res = minimize(func, ini_weights, method='SLSQP', 
                       options={'disp': True}, constraints=cons)
        locals()['weight'+str(j)][i] = res.x
        #print (res.x)

In [ ]:
class_set = list(range(num_class))
act_set = PowerSetsBinary(class_set)

act_set.remove(act_set[0]) # emptyset is not needed
act_set=sorted(act_set)


utility_matrix = np.zeros([len(act_set), len(class_set)])
tol_i = 3 

# tol_i = 0 with tol=0.5, tol_i = 1 with tol=0.6, tol_i = 2 with tol=0.7, 
# tol_i = 3 with tol=0.8, tol_i = 4 with tol=0.9

for i in range(len(act_set)):
    intersec = class_set and act_set[i]
    if len(intersec) == 1:
        utility_matrix[i, intersec] = 1
    else:
        for j in range(len(intersec)):
            utility_matrix[i, intersec[j]] = locals()[
                'weight' + str(len(intersec))][tol_i, 0]
    
#print(utility_matrix)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.8        0.8        0.         ... 0.         0.         0.        ]
 [0.68186243 0.68186243 0.68186243 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.8        0.8       ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [ ]:
import tensorflow as tf

def prepare_utility(inputs):
    nu = 0.9
    for i in range(len(utility_matrix)):
          if i==0:
                precise = tf.multiply(inputs[:, 0: num_class], utility_matrix[i], name=None)
                precise = tf.reduce_sum(precise, -1, keepdims=True)
                omega_1 = tf.multiply(inputs[:, -1], tf.reduce_max(utility_matrix[i]), name=None)
                omega_2 = tf.multiply(inputs[:, -1], tf.reduce_min(utility_matrix[i]), name=None)
                omega = tf.expand_dims(nu * omega_1 + (1 - nu)*omega_2, -1)
                omega = tf.dtypes.cast(omega, tf.float32)
                utility = tf.add(precise, omega, name=None)

          if i>=1:
                precise = tf.multiply(inputs[:, 0: num_class], utility_matrix[i], name=None)
                precise = tf.reduce_sum(precise, -1, keepdims=True)
                omega_1 = tf.multiply(inputs[:, -1], tf.reduce_max(utility_matrix[i]), name=None)
                omega_2 = tf.multiply(inputs[:, -1], tf.reduce_min(utility_matrix[i]), name=None)
                omega = tf.expand_dims(nu * omega_1 + (1 - nu)*omega_2, -1)
                omega = tf.dtypes.cast(omega, tf.float32)
                utility_i = tf.add(precise, omega, name=None)
                utility = tf.concat([utility, utility_i], -1)
    return utility

def average_utility(utility_matrix, inputs, labels, act_set):
    utility = 0
    for i in range(len(inputs)):
        x = inputs[i]
        y = labels[i]
        utility += utility_matrix[x, y]
    average_utility = utility / len(inputs)
    return average_utility

def calc_utility(pl, labels):
    util = prepare_utility(pl)
    resutls = tf.argmax(util, -1)

    imprecise_results = []
    for i in range(len(resutls)):
        act_local = resutls[i]
        set_valued_results = act_set[act_local]
        imprecise_results.append(set_valued_results)

    average_utility_imprecision = average_utility(utility_matrix, 
                                              resutls, labels, 
                                              act_set)
    return average_utility_imprecision
    #print(imprecise_results)

## Training functions

Now we can define training cycles for each of the correction schemes

In [ ]:
def get_pl(model, inputs):
    mass = model(inputs, return_mass=True)
    mass = mass[..., :-1] + mass[..., -1].repeat(10, 1).T[2]

    return mass

### Scheme 1

In [ ]:
def step_beta_scheme1(pls, ind, labels, correction, beta):
    running_epl = 0

    for i, pl in enumerate(pls):
        #print(len(labels), i, len(pl), ind)
        running_epl += calc_epl(correction(beta, pl[ind]), labels[i])

    return running_epl


def find_beta_scheme1(pls, ind, labels, correction, iters=100):
    beta = torch.rand(10, requires_grad=True)
    optimizer = torch.optim.Adam([beta], lr=1)
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                             step_size=15, gamma=0.1)

    for i in range(iters):
        optimizer.zero_grad()
        epl = step_beta_scheme1(pls, ind, labels, correction, beta)
        epl.backward()
        optimizer.step()
        lr_scheduler.step()

    beta = torch.clamp(beta, min=0., max=1.)
    epl = step_beta_scheme1(pls, ind, labels, correction, beta)

    return beta, epl

### Scheme 2

In [ ]:
def step_beta_scheme2(pls, labels, 
                      correction1, correction2, beta1, beta2):
    
    running_epl = 0

    for i, (pl1, pl2) in enumerate(pls):
        running_epl += calc_epl(correction1(beta1, pl1) * 
                                correction2(beta2, pl2), labels[i])

    return running_epl


def find_beta_scheme2(pls, labels, correction1, correction2, iters=100):
    
    beta1 = torch.rand(10, requires_grad=True)
    beta2 = torch.rand(10, requires_grad=True)
    optimizer = torch.optim.Adam([beta1, beta2], lr=1)
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                             step_size=15, gamma=0.1)

    for i in range(iters):
        optimizer.zero_grad()
        epl = step_beta_scheme2(pls, labels, correction1, correction2, 
                                beta1, beta2)
        epl.backward()
        optimizer.step()
        lr_scheduler.step()

    beta1 = torch.clamp(beta, min=0., max=1.)
    beta2 = torch.clamp(beta, min=0., max=1.)
    epl = step_beta_scheme2(pls, labels, correction1, correction2, 
                            beta1, beta2)

    return beta1, beta2, epl

### Testing function

In [ ]:
def test_correction(pls, labels, correction1, correction2, beta1, beta2):
    running_epl = 0
    util = []

    for i, (pl1, pl2) in enumerate(pls):

        with torch.no_grad():
            pl1 = correction1(beta1, pl1)
            pl2 = correction2(beta2, pl2)

        running_epl += calc_epl(pl1 * pl2, labels[i])
        util.append(calc_utility(pl1 * pl2, labels[i]))

    util = np.array(util)

    return running_epl, np.mean(util)

### Main cycle

In [ ]:
model1 = cnn
model2 = fcn
logs = []
results = defaultdict(list)
utilities = defaultdict(list)

splits = KFold(n_splits=5, shuffle=True, random_state=42)

corrections = [(CD_correction, 'CD'),
               (CR_correction, 'CR'),
               (CN_correction, 'CN')]


for fold, (train_idx, val_idx) in enumerate(
        splits.split(np.arange(len(data_corrections)))
    ):

    print('====   Fold {}   ===='.format(fold + 1))
    log = dict()

    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)

    corrections_train_loader = DataLoader(data_corrections, 
                                          batch_size=BATCH_SIZE, 
                                          sampler=train_sampler)
    
    corrections_val_loader = DataLoader(data_corrections, 
                                        batch_size=BATCH_SIZE, 
                                        sampler=val_sampler)
    
    pls_train = []
    labels_train = []
    for i, (inputs, labels) in enumerate(corrections_train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            pl1 = get_pl(model1, inputs)
            pl2 = get_pl(model2, inputs)
            pls_train.append([pl1, pl2])
            labels_train.append(labels)

    pls_val = []
    labels_val = []
    for i, (inputs, labels) in enumerate(corrections_val_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            pl1 = get_pl(model1, inputs)
            pl2 = get_pl(model2, inputs)
            pls_val.append([pl1, pl2])
            labels_val.append(labels)
    
    # no corrections
    print("No corrections")

    res, util = test_correction(pls_val, labels_val,
                          no_correction, no_correction, 0, 0)
    
       
    print("Result:", res.item())
    log['no_res'] = res
    results['no_res'].append(res)
    utilities['no_res'].append(util)
    
    print("-----------\n")
    
    # training corrections separately 
    print("Training with scheme 1")
    model1_betas = []
    model2_betas = []

    for (correction, cor_name) in corrections:
        beta, epl = find_beta_scheme1(pls_train, 0, labels_train, 
                                      correction)
        model1_betas.append({'epl':epl, 'beta':beta,
                             'name':cor_name, 'type':correction})

    for (correction, cor_name) in corrections:
        beta, epl = find_beta_scheme1(pls_train, 1, labels_train,
                                      correction)
        model2_betas.append({'epl':epl, 'beta':beta,
                             'name':cor_name, 'type':correction})

    model1_betas.sort(key=lambda x: x['epl'])
    model2_betas.sort(key=lambda x: x['epl'])

    model1_best = model1_betas[0]
    model2_best = model2_betas[0]

    print("Best correction for model 1:", model1_best['name'], 
          model1_best['beta'].tolist(), 
          "with Epl", model1_best['epl'].item())
    
    print("Best correction for model 2:", model2_best['name'], 
          model2_best['beta'].tolist(), 
          "with Epl", model2_best['epl'].item())
    

    res, util = test_correction(pls_val, labels_val,
                          model1_best['type'], model2_best['type'], 
                          model1_best['beta'], model2_best['beta'])
    
    print("Result:", res.item())
    log['scheme1_res'] = res
    results['scheme1_res'].append(res)
    utilities['scheme1_res'].append(util)

    log['scheme1_betas1'] = model1_betas
    log['scheme1_betas2'] = model2_betas
    
    print("-----------\n")
    
    # training corrections together
    print("Training with scheme 2")
    both_betas = []
    same_betas = []
    
    for (correction1, cor_name1) in corrections:
        for (correction2, cor_name2) in corrections:
            beta1, beta2, epl = find_beta_scheme2(pls_train, labels_train,
                                                correction1, correction2)
            
            both_betas.append({'epl':epl, 'beta1':beta1, 'beta2':beta2,
                               'name1':cor_name1, 'name2':cor_name2,
                               'type1':correction1, 'type2':correction2})

            if cor_name1 == cor_name2:
                same_betas.append(both_betas[-1])

    both_betas.sort(key=lambda x: x['epl'])
    both_best = both_betas[0]

    print("Best correction for both models:", 
          both_best['name1'], both_best['beta1'].tolist(), " and",
          both_best['name2'], both_best['beta2'].tolist(),
           "with Epl", both_best['epl'].item())
    
    res, util = test_correction(pls_val, labels_val, 
                          both_best['type1'], both_best['type2'],
                          both_best['beta1'], both_best['beta2'])
    
    print("Result:", res.item())
    log['scheme2_res'] = res
    results['scheme2_res'].append(res)
    utilities['scheme2_res'].append(util)

    log['scheme2_both_betas'] = both_betas
    
    print("-----------\n")

    for same in same_betas:
        print("Same correction for both models:", 
          same['name1'], same['beta1'].tolist(), " and",
          same['name2'], same['beta2'].tolist(),
           "with Epl", same['epl'].item()) 
        
        res, util = test_correction(pls_val, labels_val,
                        same['type1'], same['type2'],
                        same['beta1'], same['beta2'])
        
        print("Result:", res.item())
        log['scheme2_res_' + same['name1']] = res
        results['scheme2_res_' + same['name1']].append(res)
        utilities['scheme2_res_' + same['name1']].append(util)

    logs.append(log)
    #with open('logs.txt', 'w') as convert_file:
    #    convert_file.write(json.dumps(logs))
    
    print("===========\n\n")


data_corrections

====   Fold 1   ====
No corrections
Result: 4068.23974609375
-----------

Training with scheme 1
Best correction for model 1: CR [0.22647450864315033, 0.25417691469192505, 0.20114678144454956, 0.27195143699645996, 0.03580644726753235, 0.3709567189216614, 0.2840774953365326, 0.08410848677158356, 0.2189941108226776, 0.11360137164592743] with Epl 18047.982421875
Best correction for model 2: CR [0.01653030514717102, 0.360495388507843, 0.2530955970287323, 0.04761568456888199, 0.41582581400871277, 0.06145716458559036, 0.33622679114341736, 0.34301573038101196, 0.4202761650085449, 0.3239774703979492] with Epl 17019.126953125
Result: 5345.6806640625
-----------

Training with scheme 2
Best correction for both models: CN [0.7670679092407227, 0.8803847432136536, 0.852088212966919, 0.8351060748100281, 0.8453305959701538, 0.8142989873886108, 0.814149796962738, 0.884725034236908, 0.8634527325630188, 0.7074534893035889]  and CN [0.7670679092407227, 0.8803847432136536, 0.852088212966919, 0.83510607481

In [ ]:
for key in results:
    results[key] = np.array(results[key])
    print(key, '| mean = %.2f (%.2f)' %(np.mean(results[key]), np.std(results[key])))

no_res | mean = 4059.46 (317.27)
scheme1_res | mean = 5281.34 (93.28)
scheme2_res | mean = 2808.70 (221.63)
scheme2_res_CD | mean = 4433.44 (1101.19)
scheme2_res_CR | mean = 2981.47 (327.13)
scheme2_res_CN | mean = 3122.36 (310.45)


In [ ]:
for key in utilities:
    utilities[key] = np.array(utilities[key])
    print(key, '| mean = %.3f (%.3f)' %(np.mean(utilities[key]), np.std(utilities[key])))

no_res | mean = 0.776 (0.042)
scheme1_res | mean = 0.724 (0.054)
scheme2_res | mean = 0.760 (0.025)
scheme2_res_CD | mean = 0.776 (0.046)
scheme2_res_CR | mean = 0.776 (0.043)
scheme2_res_CN | mean = 0.713 (0.074)
